<a href="https://colab.research.google.com/github/EricCallaway/COSC_6314_Assignment_6_KNN/blob/CV/COSC_6314_KNN_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold


In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=False)

Mounted at /content/drive


Reading in the training data

In [3]:
file_path = '/content/drive/MyDrive/Colab Notebooks/Data Mining/data/diabetes.csv'
df = pd.read_csv(file_path)

In [4]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


Confirming the shape of the training data. The data is 768 rows by 9 columns.

In [5]:
df.shape

(768, 9)

Removing Zeros from columns where it is not ideal. I.e someone cannot have zero skin thickness.

In [6]:
#Replace Zeros
zero_not_accepted = ['Glucose', 'BloodPressure', 'SkinThickness', 'BMI', 'Insulin']

for column in zero_not_accepted:
  df[column] = df[column].replace(0, np.NaN)
  mean = int(df[column].mean(skipna=True))
  df[column] = df[column].replace(np.NaN, mean)

Dropping the target colunmn to create a training dataset.

In [7]:
x = df.drop(columns=['Outcome'])
x.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,6,148.0,72.0,35.0,155.0,33.6,0.627,50
1,1,85.0,66.0,29.0,155.0,26.6,0.351,31
2,8,183.0,64.0,29.0,155.0,23.3,0.672,32
3,1,89.0,66.0,23.0,94.0,28.1,0.167,21
4,0,137.0,40.0,35.0,168.0,43.1,2.288,33


In [8]:
y = df['Outcome'].values
print(y[0:10])
print(type(y))

[1 0 1 0 1 0 1 0 1 1]
<class 'numpy.ndarray'>


Creating Training and Testing sets

In [9]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=1337, stratify=y)

Perform Feature Scaling or Normalization of feature values

In [10]:
sc_x = StandardScaler()
x_train = sc_x.fit_transform(x_train)
x_test = sc_x.transform(x_test)



Creating Stratified KFold object

In [11]:
skf = StratifiedKFold(n_splits=10)
print(skf)

StratifiedKFold(n_splits=10, random_state=None, shuffle=False)


Function to get the accuracy score.

In [12]:
def get_score(model, x_train, x_test, y_train, y_test):
    model.fit(x_train, y_train)
    return model.score(x_test, y_test)

Bulding the Classifier object

In [13]:
classifier = KNeighborsClassifier(n_neighbors=11, p=2, metric='euclidean')

Implementing Stratified KFold on the dataset.

In [14]:
scores = []

for train_index, test_index in skf.split(x,y):
  print('TRAIN:', train_index, 'TEST:', test_index)
  x_train_, x_test_ = x.iloc[train_index], x.iloc[test_index]
  y_train_, y_test_ = y[train_index], y[test_index]
  
  scores.append(get_score(classifier, x_train, x_test, y_train, y_test))

TRAIN: [ 61  64  66  70  72  78  83  84  85  86  87  88  89  90  91  92  93  94
  95  96  97  98  99 100 101 102 103 104 105 106 107 108 109 110 111 112
 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130
 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148
 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166
 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184
 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202
 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220
 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238
 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256
 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274
 275 276 277 278 279 280 281 282 283 284 285 286 287 288 289 290 291 292
 293 294 295 296 297 298 299 300 301 302 303 304 305 306 307 308 309 310
 311 312 313 314 315 316 317 318 319 320 321

In [15]:
np.average(scores)

0.7857142857142857

Define the Model

Fit the Model

In [16]:
cv_scores = cross_val_score(classifier, x, y, cv=10)

In [17]:
print(cv_scores)
print('cv_scores mean:{}'.format(np.mean(cv_scores)))

[0.68831169 0.75324675 0.68831169 0.66233766 0.71428571 0.72727273
 0.76623377 0.79220779 0.75       0.80263158]
cv_scores mean:0.7344839371155161


In [18]:
classifier.fit(x_train, y_train)

KNeighborsClassifier(metric='euclidean', n_neighbors=11)

Predict the test set results

In [19]:
y_pred = classifier.predict(x_test)
y_pred

array([1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0,
       0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0,
       1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1])

Evaluate Model

In [20]:
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[83 17]
 [16 38]]


In [21]:
cr = classification_report(y_true=y_test, y_pred=y_pred, target_names=['No Diabetes', 'Diabetes'])
print(cr)

              precision    recall  f1-score   support

 No Diabetes       0.84      0.83      0.83       100
    Diabetes       0.69      0.70      0.70        54

    accuracy                           0.79       154
   macro avg       0.76      0.77      0.77       154
weighted avg       0.79      0.79      0.79       154



In [22]:
print(f1_score(y_pred, y_test))

0.6972477064220184
